In [ ]:
import os
import numpy as np
import pickle
from utils import utils_gn, utils_dgrd
import importlib
importlib.reload(utils_gn)
importlib.reload(utils_dgrd)

Here, we remove all cells whose end of life (EOL) is greater than 1200, a data prpcessing step. We also remove cells that belong to batches 4 to 7 as they are cycled pass what can be regarded as EOL. Also, we will split the *raw data* into train and test cells, and dump them as pickle files. Although this might be can considered as too early, it is the best practice to carry out exploration and transformation only on the train data. This will curb *peeping into the test data*, *over-fitting* and *biasedness*.

In [ ]:
# Load all the batches 
batch = utils_gn.read_data('data_101cycles_ir.pkl')

In [ ]:
# Remove batteries with more than 1200 cycle life and those belong to batches 4 to 7
odd_cells = []

for cell in batch.keys():
    Qd = batch[cell]['summary']['QDischarge']
    Qd_eol = Qd >= .88   # eol is defined as cycle number at 80% of nominal capacity
    Qd = Qd[Qd_eol]
    if np.any([len(Qd) > 1200, cell[:2] in ('b4', 'b5', 'b6', 'b7')]):
        odd_cells.append(cell) 
odd_cells

In [ ]:
# Check the number cells left 
new_batch = {k: batch[k] for k in batch.keys() if k not in odd_cells}
len(new_batch.keys())

In [ ]:

# Split the data into 70-30% train-test and dump in a pickle file
train, test = utils_gn.split_train_test_by_id(new_batch, test_ratio=0.3)

with open(os.path.join("data", "train_1238.pkl"), "wb") as fp:
    pickle.dump(train, fp)

with open(os.path.join("data", "test_1238.pkl"), "wb") as fp:
    pickle.dump(test, fp)

In [ ]:
# Get the true labels for the test data
true_test_labels = utils_dgrd.create_knee_elbow_data(test)
with open(os.path.join("data", "true_test_labels_1238.pkl"), "wb") as fp:
    pickle.dump(true_test_labels, fp)

In [ ]:
# Get the true labels for the train data
true_train_labels = utils_dgrd.create_knee_elbow_data(train)
with open(os.path.join("data", "true_train_labels_1238.pkl"), "wb") as fp:
    pickle.dump(true_train_labels, fp)